In [670]:
import pandas as pd
import numpy as np

In [68]:
#df = pd.read_excel('test_df.xlsx', index_col = 0)
t = pd.read_excel('test.xlsx', sheet_name = 'Sheet2', index_col = 0)

In [69]:
def days_buying_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1].reset_index(drop = True)
    df['row_1_zero_index'] = (df.index + 1) * (df['c'] == 0)
    days_buying = df.loc[df['row_1_zero_index'] > 0, 'row_1_zero_index'].min() - 1
    if np.isnan(days_buying):
        days_buying = df.shape[0]        
    return days_buying

In [70]:
def get_days_buying(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'days_buying'] = days_buying_by_row(df.loc[r])
    return tmp

In [71]:
get_days_buying(t)

,days_buying
row_0,0.0
row_1,3.0
row_2,0.0
row_3,9.0
row_4,0.0
row_5,0.0
row_6,1.0


In [72]:
def zero_days_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1].reset_index(drop = True)
    df['row_zero_index'] = (df.index + 1) * (df['c'] != 0)
    res = df.loc[df['row_zero_index'] != 0, 'row_zero_index'].min() - 1
    if np.isnan(res):
        res = df.shape[0]        
        
    return res

zero_days_by_row(t.iloc[0])

9

In [73]:
def get_zero_days(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'zero_days'] = zero_days(df.loc[r])
    return tmp

get_zero_days(t)

,zero_days
row_0,9.0
row_1,0.0
row_2,1.0
row_3,0.0
row_4,4.0
row_5,2.0
row_6,0.0


In [74]:
def zero_days_before_yesterday_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1].iloc[1:, :].reset_index(drop = True)
    df['row_zero_index'] = (df.index + 1) * (df['c'] != 0)
    res = df.loc[df['row_zero_index'] != 0, 'row_zero_index'].min() - 1
    if np.isnan(res):
        res = df.shape[0]        
        
    return res

zero_days_before_yesterday_by_row(t.iloc[0])

8

In [75]:
def get_zero_days_before_yesterday(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'zero_days_before_yesterday'] = zero_days_before_yesterday_by_row(df.loc[r])
    return tmp

get_zero_days_before_yesterday(t)

,zero_days_before_yesterday
row_0,8.0
row_1,0.0
row_2,0.0
row_3,0.0
row_4,3.0
row_5,1.0
row_6,8.0


In [76]:
def days_buying_before_yesterday_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1]
    df = df.iloc[1:, :].reset_index(drop = True)
    df['row_1_zero_index'] = (df.index + 1) * (df['c'] == 0)
    days_buying = df.loc[df['row_1_zero_index'] > 0, 'row_1_zero_index'].min() - 1
    if np.isnan(days_buying):
        days_buying = df.shape[0]        
    return days_buying

In [77]:
def get_days_buying_before_yesterday_by_row(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'days_buying_before_yesterday'] = days_buying_before_yesterday_by_row(df.loc[r])
    return tmp

In [78]:
get_days_buying_before_yesterday_by_row(t)

,days_buying_before_yesterday
row_0,0.0
row_1,2.0
row_2,3.0
row_3,8.0
row_4,0.0
row_5,0.0
row_6,0.0


In [95]:
def avg_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'}).iloc[::-1].iloc[1:, :]
    avg = df['c'].mean()
    return avg

avg_by_row(t.iloc[6])

0.0

In [96]:
def get_avg_by_row(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'avg_by_row'] = avg_by_row(df.loc[r])
    return tmp

get_avg_by_row(t)

,avg_by_row
row_0,0.000
row_1,9.250
row_2,21.375
row_3,23.250
row_4,14.250
row_5,17.250
row_6,0.000


In [98]:
def avg_by_non_zero_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'}).iloc[::-1].iloc[1:, :]
    avg = df.loc[df['c']>0, 'c'].mean()
    if np.isnan(avg):
        avg = 0
    return avg
avg_by_non_zero_by_row(t.iloc[0])

0

In [99]:
def get_avg_by_non_zero_by_row(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'avg_by_nonzero_row'] = avg_by_non_zero_by_row(df.loc[r])
    return tmp

get_avg_by_non_zero_by_row(t)

,avg_by_nonzero_row
row_0,0.000000
row_1,14.800000
row_2,24.428571
row_3,23.250000
row_4,22.800000
row_5,23.000000
row_6,0.000000


In [83]:
#is zero yesterday

In [84]:
def is_zero_yesterday(df):
    tmp = pd.DataFrame()
    tmp['is_zero_yesterday'] = df.iloc[:, -1] == 0
    return tmp['is_zero_yesterday']

In [85]:
t

,1,2,3,4,5,6,7,8,9
row_0,0,0,0,0,0,0,0,0,0
row_1,0,12,13,14,0,0,17,18,7
row_2,21,22,23,24,0,26,27,28,0
row_3,21,22,23,24,24,24,24,24,24
row_4,21,22,23,24,24,0,0,0,0
row_5,21,22,23,24,24,0,24,0,0
row_6,0,0,0,0,0,0,0,0,1


In [86]:
is_zero_yesterday(t)

row_0     True
row_1    False
row_2     True
row_3    False
row_4     True
row_5     True
row_6    False
Name: is_zero_yesterday, dtype: bool

In [120]:
def share_of_days_buying_before_yesterday(df):
    df = df.iloc[:, :-1].T
    tmp = df > 0
    tmp.loc['share_non_zero_days'] = tmp.mean()
    return tmp.T['share_non_zero_days']
    
share_of_days_buying_before_yesterday(t)

row_0      0.0
row_1    0.625
row_2    0.875
row_3      1.0
row_4    0.625
row_5     0.75
row_6      0.0
Name: share_non_zero_days, dtype: object

In [87]:
#alerts

In [121]:
tt = (
    t
    .join(get_days_buying(t))
    .join(get_days_buying_before_yesterday_by_row(t))
    .join(get_avg_by_row(t))
    .join(get_avg_by_non_zero_by_row(t))
    .join(is_zero_yesterday(t))
    .join(get_zero_days(t))
    .join(get_zero_days_before_yesterday(t))
    .join(share_of_days_buying_before_yesterday(t))
)

tt

,1,2,3,4,5,6,7,8,9,days_buying,days_buying_before_yesterday,avg_by_row,avg_by_nonzero_row,is_zero_yesterday,zero_days,zero_days_before_yesterday,share_non_zero_days
row_0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000,0.000000,True,9.0,8.0,0.0
row_1,0,12,13,14,0,0,17,18,7,3.0,2.0,9.250,14.800000,False,0.0,0.0,0.625
row_2,21,22,23,24,0,26,27,28,0,0.0,3.0,21.375,24.428571,True,1.0,0.0,0.875
row_3,21,22,23,24,24,24,24,24,24,9.0,8.0,23.250,23.250000,False,0.0,0.0,1.0
row_4,21,22,23,24,24,0,0,0,0,0.0,0.0,14.250,22.800000,True,4.0,3.0,0.625
row_5,21,22,23,24,24,0,24,0,0,0.0,0.0,17.250,23.000000,True,2.0,1.0,0.75
row_6,0,0,0,0,0,0,0,0,1,1.0,0.0,0.000,0.000000,False,0.0,8.0,0.0


In [20]:
#set rules

In [153]:
#### partner X stop bying

print('Вывожу статистику партнёров, которые ранее были, а вчера исчезли:')
stop_buying_df = tt.query('is_zero_yesterday == True & zero_days_before_yesterday == 0')

for r in stop_buying_df.index:
    days_buying_before_yesterday = tt.loc[[r], 'days_buying_before_yesterday'][0]
    share_non_zero_days = tt.loc[[r], 'share_non_zero_days'][0]
    avg_by_nonzero_row = tt.loc[[r], 'avg_by_nonzero_row'][0]
   
    print(
        'Партнёр ' + r + ', работал ' + str(int(days_buying_before_yesterday)) + 
        ' дня до того, как пропасть. За период брал ' + str(share_non_zero_days * 100) + 
        '% дней в среднем по ' + str(round(avg_by_nonzero_row, 1)) + ' за день.'
    )

print('Job done!!!')

Вывожу статистику партнёров, которые ранее были, а вчера исчезли:
Партнёр row_2, работал 3 дня до того, как пропасть. За период брал 87.5% дней в среднем по 24.4 за день.
